In [38]:

from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from sascorer import calculateScore

In [41]:
def tg_based_scoring(tg):
    if tg >= 50 and tg <= 100:
        return round(1 - (abs(tg-75)/25),2)
    return 0.0

def er_based_scoring(er):
    if er >150 and er <300:
        return round(1 - (abs(er-225)/75),2)
    return 0.0

def property_based_reward_scoring(generated_smiles):
    tg= 70 # predict ( generated_smiles)
    er=200 # predict ( geneated smiles)
    tg_score=tg_based_scoring(tg)
    er_score=er_based_scoring(er)
    print(tg_score,er_score)
    return round((tg_score+er_score)/2,2)
#property_based_scoring(70, 230)  

In [42]:
def synthesis_reward_scoring(geneated_smiles):
    polymer_smiles = "CCOCCOC=C" #geneated_smile
    polymer_molecule = Chem.MolFromSmiles(polymer_smiles)
    sa_score_polymer = calculateScore(polymer_molecule)
    return round((10-sa_score_polymer)/10, 2)


In [43]:
epoxy_smarts = Chem.MolFromSmarts('C1OC1') 
primary_amine_smarts = Chem.MolFromSmarts('[NX3;H2]')
secondary_amine_smarts = Chem.MolFromSmarts('[NX3;H1]([C])')

def check_and_count_group_presence(monomer_smiles, functional_group_smarts):
    monomer_molecule = Chem.MolFromSmiles(monomer_smiles)
    if monomer_molecule:
        return len(monomer_molecule.GetSubstructMatches(functional_group_smarts))
    return 0

def functional_group_reward_score(generated_smiles):
    monomer1_smiles = 'C1OC1CC3OC3' # extract from generated smiles
    monomer2_smiles = 'NCCN' #extract from geneated smiles
    monomer1_epoxy_count = check_and_count_group_presence(monomer1_smiles, epoxy_smarts)
    monomer2_primary_amine_count = check_and_count_group_presence(monomer2_smiles, primary_amine_smarts)
    monomer2_secondary_amine_count = check_and_count_group_presence(monomer2_smiles, secondary_amine_smarts)
    monomer2_total_amine_count = monomer2_primary_amine_count + monomer2_secondary_amine_count

    # Determine if the criteria are met
    monomer1_meets_criteria = monomer1_epoxy_count >= 2
    monomer2_meets_criteria = monomer2_total_amine_count >= 2
    
    # Reward score calculation
    if monomer1_meets_criteria and monomer2_meets_criteria:
        reward_score = 1.0  # Maximum reward if both conditions are met
    elif monomer1_meets_criteria or monomer2_meets_criteria:
        reward_score = 0.5  # Partial reward if only one condition is met
    else:
        reward_score = 0.0  # No reward if neither condition is met

    return reward_score


reward = functional_group_reward_score(generated_smiles)

print(f"Reward score for the monomers: {reward}")    

Reward score for the monomers: 1.0


In [45]:
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

# Example: Assume we have a list of known polymer SMILES from a database
known_polymer_smiles = [
    'C1COC1',          # Example of a polymer with an epoxy group
    'NCCNCC',          # Example of a polymer with primary amines
    'CC(C)COC(C)C',    # Example polymer with branching
    # Add more known polymers here...
]


known_polymers = [Chem.MolFromSmiles(smiles) for smiles in known_polymer_smiles]
known_fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in known_polymers]

def compute_similarity_score(generated_smiles):
    generated_molecule = Chem.MolFromSmiles(generated_smiles)
    if not generated_molecule:
        return 0.0  # Return a similarity score of 0 if the molecule is invalid
    
    generated_fingerprint = AllChem.GetMorganFingerprintAsBitVect(generated_molecule, 2, nBits=2048)
    
    max_similarity = 0.0
    for known_fp in known_fingerprints:
        similarity = DataStructs.TanimotoSimilarity(generated_fingerprint, known_fp)
        max_similarity = max(max_similarity, similarity)
    
    return max_similarity

def similarity_reward_scoring(generated_smiles):
   
    similarity_score = compute_similarity_score(generated_smiles)
    print(similarity_score)
    if similarity_score >= 0.8:
        reward_score = 1.0  
    elif similarity_score >= 0.5:
        reward_score = 0.5  
    else:
        reward_score = 0.1 
    
    return reward_score

generated_smiles = 'CC(C)C2OC2(C)C'  # Example of a generated polymer

# Calculate the reward score
reward = similarity_reward_scoring(generated_smiles)

print(f"Reward score for the generated polymer: {reward}")


0.125
Reward score for the generated polymer: 0.1


In [47]:
def min_max_normalize(score, min_value, max_value):
    if max_value==min_value:
        return 0.0
    return (score - min_value) / (max_value - min_value)

def calculate_composite_reward(generated_smiles, w1=0.3, w2=0.3, w3=0.2, w4=0.2):
   
    property_min, property_max = 0.0, 1.0
    fg_min, fg_max = 0.0, 1.0
    sa_min, sa_max = 0.0, 1.0  # Example SA score range
    similarity_min, similarity_max = 0.0, 1.0

    
    property_score = property_based_reward_scoring(generated_smiles)
    synthesis_score = synthesis_reward_scoring(generated_smiles)
    functional_group_score = functional_group_reward_score(generated_smiles)
    similarity_score = similarity_reward_scoring(generated_smiles)
    
    # Normalize the scores
    norm_property_score = min_max_normalize(property_score, property_min, property_max)
    norm_fg_score = min_max_normalize(functional_group_score, fg_min, fg_max)
    norm_sa_score = min_max_normalize(synthesis_score, sa_min, sa_max)
    norm_similarity_score = min_max_normalize(similarity_score, similarity_min, similarity_max)

    # Combine scores using the assigned weights
    total_reward_score = (w1 * norm_property_score +
                          w2 * norm_fg_score +
                          w3 * norm_sa_score +
                          w4 * norm_similarity_score)
    
    return total_reward_score
calculate_composite_reward('CC(C)C2OC2(C)C')

0.8 0.67
0.125


0.6900000000000001

In [27]:
from rdkit import Chem
from rdkit.Chem import Descriptors

# Define the SMILES strings for the monomers
monomer1_smiles = 'OCC(O)(COC1CO1)COC2CO2'#'NCCN'  # Example monomer with a vinyl and carboxyl group
monomer2_smiles = 'CCNc1ccc(cc1)Cc2ccc(N=C=O)cc2'#'C2OC2CCOC1OC1'  # Example monomer with an alcohol group

# Convert SMILES to RDKit Molecule objects
monomer1 = Chem.MolFromSmiles(monomer1_smiles)
monomer2 = Chem.MolFromSmiles(monomer2_smiles)

# Identify functional groups using predefined SMARTS patterns
functional_groups = {
    'Hydroxyl': '[OX2H]',  # Alcohol -OH group
    'Carboxyl': '[CX3](=O)[OX2H1]',  # Carboxylic acid -COOH group
    'Amine': '[NX3;H2,H1;!$(NC=O)]',  # Primary or secondary amine -NH2 or -NH-
    'Isocyanate': '[NX1]=[CX2]=[OX1]',  # Isocyanate -N=C=O group
    'Vinyl': '[CX3]=[CX2]',  # Vinyl group -CH=CH2
    #'Epoxide': '[OX2r3]',  # Epoxy group (three-membered ring with oxygen)
    'Epoxy': '[OX2r3][CX4][CX4]',  # Epoxy group (three-membered ring with oxygen and two carbons)
    'Aldehyde': '[CX3H1](=O)[#6]',  # Aldehyde -CHO group
    'Ketone': '[#6][CX3](=O)[#6]',  # Ketone -C(=O)- group
    'Ester': '[CX3](=O)[OX2][#6]',  # Ester -COOR group
    'Thiol': '[#16X2H]',  # Thiol -SH group
    'Thioester': '[CX3](=O)[SX2][#6]',  # Thioester -COSR group
    'Ether': '[OD2]([#6])[#6]',  # Ether -O- group
    'Anhydride': '[CX3](=O)O[CX3](=O)',  # Anhydride -CO-O-CO- group
    'Acyl Chloride': '[CX3](=O)Cl',  # Acyl chloride -COCl group
    'Sulfonic Acid': '[SX4](=O)(=O)([OX2H])O',  # Sulfonic acid -SO3H group
    'Sulfonamide': '[NX3][SX4](=O)(=O)[#6]',  # Sulfonamide -SO2NR group
    'Phosphate': '[PX4](=O)([OX1-])(O[H1,O])',  # Phosphate group -PO4
    'Nitrile': '[CX2]#N',  # Nitrile -C≡N group
    'Phenol': '[cX3][OX2H]',  # Phenol -ArOH group
    'Azo': '[NX2]=[NX2]',  # Azo -N=N- group
    'Azide': '[NX3]([NX1]=[NX1])',  # Azide -N3 group
    'Amide': '[NX3][CX3](=O)[#6]',  # Amide -CONH- group
    'Imide': '[NX3]([CX3](=O))[CX3](=O)',  # Imide -CONHCO- group
    'Peroxide': '[OX2][OX2]',  # Peroxide -O-O- group
    'Carbonate': '[CX3](=O)[OX2][CX3](=O)',  # Carbonate -O(CO)O- group
    'Isothiocyanate': '[NX1]=[CX2]=[SX1]',  # Isothiocyanate -N=C=S group
    'Carbamate': '[NX3][CX3](=O)[OX2][#6]',  # Carbamate -OC(=O)NR- group
    'Boronate': '[BX3]([OX2])[OX2]',  # Boronate group -B(OR)2
    'Triazole': '[nX2]1[nX2][nX2]1',  # Triazole group (five-membered ring with three nitrogens)
    'Tetrazole': '[nX2]1[nX2][nX2][nX2]1',  # Tetrazole group (five-membered ring with four nitrogens)
    'Phosphonate': '[PX3](=O)(O[H1,O])([#6])',  # Phosphonate group -PO(OR)2
    # Add more as needed
}


# Updated function to check for functional groups in a monomer
def find_functional_groups(molecule, fg_dict):
    #found_groups = {}
    found_groups = []
    for name, smarts in fg_dict.items():
        pattern = Chem.MolFromSmarts(smarts)
        if molecule.HasSubstructMatch(pattern):
            #found_groups[name] = True
            found_groups.append(name)
    return found_groups

# Analyze monomers for functional groups
monomer1_groups = find_functional_groups(monomer1, functional_groups)
monomer2_groups = find_functional_groups(monomer2, functional_groups)

# Print results
print(f'Monomer 1 Functional Groups: {monomer1_groups}')
print(f'Monomer 2 Functional Groups: {monomer2_groups}')

# Define the full list of functional group reactivity pairs
reactivity_pairs = [
    ('Hydroxyl', 'Carboxyl', 'Esterification', 'Ester (-COOR)'),
    ('Hydroxyl', 'Isocyanate', 'Urethane Formation', 'Urethane (-NHCOO-)'),
    ('Hydroxyl', 'Acyl Chloride', 'Esterification', 'Ester (-COOR)'),
    ('Hydroxyl', 'Anhydride', 'Esterification and Acid Formation', 'Ester (-COOR) + Carboxylic Acid'),
    ('Hydroxyl', 'Epoxy', 'Nucleophilic Ring Opening', 'Beta-Hydroxy Ether'),
    ('Hydroxyl', 'Isothiocyanate', 'Thiocarbamate Formation', 'Thiocarbamate'),
    ('Hydroxyl', 'Aldehyde', 'Hemiacetal Formation', 'Hemiacetal (-C(OH)(OR)-)'),
    ('Amine', 'Carboxyl', 'Amidation', 'Amide (-CONH-)'),
    ('Amine', 'Acyl Chloride', 'Amidation', 'Amide (-CONH-)'),
    ('Amine', 'Isocyanate', 'Urea Formation', 'Urea (-NHCONH-)'),
    ('Amine', 'Anhydride', 'Amidation and Acid Formation', 'Amide (-CONH-) + Carboxylic Acid'),
    ('Amine', 'Epoxy', 'Nucleophilic Ring Opening', 'Beta-Amino Alcohol'),
    ('Amine', 'Aldehyde', 'Schiff Base Formation', 'Imine (=N-)'),
    ('Amine', 'Ketone', 'Imine Formation', 'Imine (=N-)'),
    ('Amine', 'Isothiocyanate', 'Thiourea Formation', 'Thiourea (-NHCSNH-)'),
    ('Carboxyl', 'Hydroxyl', 'Esterification', 'Ester (-COOR)'),
    ('Carboxyl', 'Amine', 'Amidation', 'Amide (-CONH-)'),
    ('Carboxyl', 'Epoxy', 'Nucleophilic Ring Opening', 'Beta-Hydroxy Ester'),
    ('Carboxyl', 'Isocyanate', 'Carbamic Acid Formation', 'Carbamate (-COONH-)'),
    ('Carboxyl', 'Acyl Chloride', 'Acid Chloride Formation', 'Acid Chloride (-COCl)'),
    ('Carboxyl', 'Anhydride', 'Formation of Acids', 'Carboxylic Acid'),
    ('Isocyanate', 'Hydroxyl', 'Urethane Formation', 'Urethane (-NHCOO-)'),
    ('Isocyanate', 'Amine', 'Urea Formation', 'Urea (-NHCONH-)'),
    ('Isocyanate', 'Water', 'Hydrolysis', 'Amine (-NH2) and CO2'),
    ('Isocyanate', 'Epoxy', 'Epoxy-Isocyanate Reaction', 'Isocyanate Polymer'),
    ('Epoxy', 'Amine', 'Nucleophilic Ring Opening', 'Beta-Amino Alcohol'),
    ('Epoxy', 'Carboxyl', 'Nucleophilic Ring Opening', 'Beta-Hydroxy Ester'),
    ('Epoxy', 'Hydroxyl', 'Nucleophilic Ring Opening', 'Beta-Hydroxy Ether'),
    ('Epoxy', 'Thiol', 'Thiol-Epoxy Reaction', 'Thioether or Beta-Mercapto Alcohol'),
    ('Epoxy', 'Acyl Chloride', 'Nucleophilic Ring Opening', 'Halohydrin'),
    ('Aldehyde', 'Hydroxyl', 'Acetal/Ketal Formation', 'Acetal (C(OR)2) or Hemiacetal'),
    ('Aldehyde', 'Amine', 'Schiff Base Formation', 'Imine (=N-)'),
    ('Aldehyde', 'Cyanide', 'Cyanohydrin Formation', 'Cyanohydrin (-C(OH)CN)'),
    ('Aldehyde', 'Nucleophiles (e.g., RMgX)', 'Nucleophilic Addition', 'Alcohol (-OH)'),
    ('Ketone', 'Amine', 'Imine Formation', 'Imine (=N-)'),
    ('Ketone', 'Hydroxyl', 'Hemiacetal/Ketal Formation', 'Hemiacetal or Ketal'),
    ('Ketone', 'Nucleophiles (e.g., RMgX)', 'Nucleophilic Addition', 'Alcohol (-OH)'),
    ('Thiol', 'Epoxy', 'Thiol-Epoxy Reaction (Ring Opening)', 'Thioether or Beta-Mercapto Alcohol'),
    ('Thiol', 'Vinyl', 'Thiol-Ene Click Chemistry', 'Thioether'),
    ('Thiol', 'Disulfide', 'Disulfide Exchange Reaction', 'New Disulfide Bonds'),
    ('Vinyl', 'Radical Initiators', 'Radical Polymerization', 'Polyvinyl Chain'),
    ('Vinyl', 'Diene (Conjugated Diene)', 'Diels-Alder Cycloaddition', 'Cyclohexene Derivative'),
    ('Acyl Chloride', 'Hydroxyl', 'Esterification', 'Ester (-COOR)'),
    ('Acyl Chloride', 'Amine', 'Amidation', 'Amide (-CONH-)'),
    ('Anhydride', 'Hydroxyl', 'Esterification and Acid Formation', 'Ester and Carboxylic Acid'),
    ('Anhydride', 'Amine', 'Amidation and Acid Formation', 'Amide and Carboxylic Acid'),
    ('Nitrile', 'Water', 'Hydrolysis', 'Amide or Carboxylic Acid'),
    ('Nitrile', 'Amine', 'Nucleophilic Addition', 'Amidine'),
    ('Phenol', 'Acyl Chloride', 'Esterification', 'Ester (-COOR)'),
    ('Azo', 'Reducing Agents', 'Reduction', 'Amines (-NH2)'),
    ('Triazole', 'Alkynes', 'Azide-Alkyne Cycloaddition (Click Reaction)', '1,2,3-Triazole'),
    ('Carbamate', 'Hydroxyl', 'Transesterification', 'Carbamate'),
    ('Isothiocyanate', 'Amine', 'Thiourea Formation', 'Thiourea (-NHCSNH-)'),
    ('Boronate', 'Hydroxyl', 'Boronic Ester Formation', 'Boronic Ester (-B(OR)3)'),
    ('Boronate', 'Amine', 'Boron-Nitrogen Complex Formation', 'Boronate Complex'),
]

# Function to check for potential reactions between two monomers
def check_potential_reactions(monomer1_groups, monomer2_groups, reactivity_pairs):
    potential_reactions = []
    for fg1, fg2, reaction, product in reactivity_pairs:
        if fg1 in monomer1_groups and fg2 in monomer2_groups:
            potential_reactions.append((fg1, fg2, reaction, product))
        elif fg2 in monomer1_groups and fg1 in monomer2_groups:
            potential_reactions.append((fg2, fg1, reaction, product))
            
        # if monomer1_groups.get(fg1, False) and monomer2_groups.get(fg2, False):
        #     potential_reactions.append((fg1, fg2, reaction, product))
        # elif monomer1_groups.get(fg2, False) and monomer2_groups.get(fg1, False):
        #     potential_reactions.append((fg2, fg1, reaction, product))
    return set(potential_reactions)

# Example usage
monomer1_reactions = check_potential_reactions(monomer1_groups, monomer2_groups, reactivity_pairs)

# Display potential reactions
if monomer1_reactions:
    print("Potential reactions between Monomer 1 and Monomer 2:")
    for fg1, fg2, reaction, product in monomer1_reactions:
        print(f"- {fg1} group in Monomer 1 and {fg2} group in Monomer 2: {reaction} forming {product}")
else:
    print("No significant reactions predicted between Monomer 1 and Monomer 2.")



Monomer 1 Functional Groups: ['Hydroxyl', 'Epoxy', 'Ether']
Monomer 2 Functional Groups: ['Amine']
Potential reactions between Monomer 1 and Monomer 2:
- Epoxy group in Monomer 1 and Amine group in Monomer 2: Nucleophilic Ring Opening forming Beta-Amino Alcohol


In [46]:
reaction_weights = {
    'Urethane Formation': 2.0,
    'Epoxy Ring Opening': 2.5,
    'Thiol-Epoxy Reaction': 2.5,
    'Amidation': 1.5,
    'Anhydride Reaction with Hydroxyl or Amine': 1.8,
    'Esterification': 1.2,
    'Isocyanate Reactions with Water': 1.0,
    'Acyl Chloride Reactions with Amine or Hydroxyl': 1.3,
    'Diels-Alder Cycloaddition': 2.0,
    'Schiff Base Formation': 1.5,
    'Disulfide Exchange': 2.0,
    'Triazole Formation (Click Chemistry)': 2.2,
    'Cyanate Ester Formation': 2.5,
    'Nucleophilic Ring Opening':3.0
}

# Function to calculate a reward score based on potential reactions
def reaction_based_reward_scoring(monomer1_groups, monomer2_groups, reactivity_pairs, weight_dict=None):
    potential_reactions = check_potential_reactions(monomer1_groups, monomer2_groups, reactivity_pairs)
    print(potential_reactions)
    
    reward_score = 0.0

    for fg1, fg2, reaction, product in potential_reactions:
        # Add the weight for each detected reaction
        reward_score += weight_dict.get(reaction, 1.0)

    # Calculate the maximum possible score for the given monomers
    max_possible_score = reaction_weights.get(max(reaction_weights, key=reaction_weights.get))
    print(max_possible_score)

    # Normalize the score
    if max_possible_score > 0:
        normalized_reward = reward_score / max_possible_score
    else:
        normalized_reward = 0.0 

    return normalized_reward


# Calculate the reward score
reward = calculate_reward(monomer1_groups, monomer2_groups, reactivity_pairs, weight_dict=reaction_weights)

print(f"Reward score for the generated monomers: {reward:.2f}")



{('Epoxy', 'Amine', 'Nucleophilic Ring Opening', 'Beta-Amino Alcohol')}
3.0
Reward score for the generated monomers: 1.00


In [20]:
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

# Example known polymers database (SMILES strings)
known_polymer_smiles = [
    'C1COC1CCN',       # Example known polymer
    'CC(C)COC(C)CNCC', # Example known polymer with amine and ether groups
    'O=C(NCCN)N1CCOCC1',
    'C1OC1CCC2OC2NCCN',
    'C1COC1CNCCN',  # Epoxy-amine cured polymer (similar to a network polymer formed by epichlorohydrin and ethylene diamine)
    'CC(C)COC(C)CNCC',  # Another possible polymer
    'NCCOCCNCCOCCN'
    # Add more known polymers here...
]

# Convert known polymer SMILES to RDKit molecules and compute fingerprints
known_polymers = [Chem.MolFromSmiles(smiles) for smiles in known_polymer_smiles]
known_fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in known_polymers]


In [21]:
def compute_combined_fingerprint(monomer1_smiles, monomer2_smiles):
    """
    Function to compute the combined fingerprint of two monomers forming a TSMP.
    """
    # Combine the monomers' SMILES into a single string
    combined_smiles = monomer1_smiles + '.' + monomer2_smiles  # Combine using '.' for mixtures
    
    # Convert to RDKit molecule
    combined_molecule = Chem.MolFromSmiles(combined_smiles)
    if not combined_molecule:
        return None  # Return None if the molecule is invalid

    # Compute fingerprint for the combined molecule
    combined_fingerprint = AllChem.GetMorganFingerprintAsBitVect(combined_molecule, 2, nBits=2048)
    return combined_fingerprint

def compute_similarity_score_for_combined(monomer1_smiles, monomer2_smiles):
    """
    Function to compute the similarity score of a generated two-monomer-based TSMP
    with respect to known polymers in the database.
    """
    # Compute the combined fingerprint of the two monomers
    combined_fingerprint = compute_combined_fingerprint(monomer1_smiles, monomer2_smiles)
    if not combined_fingerprint:
        return 0.0  # Return a similarity score of 0 if the combined molecule is invalid

    # Compute the maximum similarity score against the known polymer database
    max_similarity = 0.0
    for known_fp in known_fingerprints:
        similarity = DataStructs.TanimotoSimilarity(combined_fingerprint, known_fp)
        max_similarity = max(max_similarity, similarity)

    return max_similarity


def database_comparison_reward_for_combined(monomer1_smiles, monomer2_smiles, threshold=0.7):
    """
    Reward function based on the Tanimoto similarity of a generated two-monomer-based TSMP
    to known structures in the database.
    """
    similarity_score = compute_similarity_score_for_combined(monomer1_smiles, monomer2_smiles)
    print(similarity_score)

    # Define reward based on similarity score
    if similarity_score >= threshold:
        reward = 1.0  # High reward for high similarity
    elif similarity_score >= 0.5:
        reward = 0.5  # Moderate reward for medium similarity
    else:
        reward = 0.1  # Low reward for low similarity

    return reward

# Example two monomers for a generated TSMP
monomer1_smiles = 'C1COC1Cl'  # Epoxy-containing monomer
monomer2_smiles = 'NCCN'    # Amine-containing monomer

# Calculate the reward based on database comparison
reward = database_comparison_reward_for_combined(monomer1_smiles, monomer2_smiles)

print(f"Reward score for the two-monomer-based TSMP: {reward}")

0.5
Reward score for the two-monomer-based TSMP: 0.5


Reward score for the two-monomer-based TSMP: 0.1
